In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_DIR = "../src/data/"

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/train", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")

val_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/val", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")

test_generator = datagen.flow_from_directory(
    f"{DATASET_DIR}/test", target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical")


Found 4359 images belonging to 5 classes.
Found 932 images belonging to 5 classes.
Found 939 images belonging to 5 classes.


In [3]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # On gèle les couches de ResNet

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)  # 5 classes

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Epoch 1/10
137/137 [==============================] - 854s 6s/step - loss: 1.7411 - accuracy: 0.2842 - val_loss: 1.4204 - val_accuracy: 0.3476
Epoch 2/10
137/137 [==============================] - 796s 6s/step - loss: 1.6090 - accuracy: 0.3161 - val_loss: 1.4184 - val_accuracy: 0.3519
Epoch 3/10
 22/137 [===>..........................] - ETA: 12:31 - loss: 1.6039 - accuracy: 0.3111

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Accuracy sur test : {test_acc:.2f}")